In [1]:
# dependencies
import pandas as pd 
import requests
import json
import gmaps
import datetime

from config import gkey 

In [2]:
raw_data = pd.read_csv("../Resources/Twitter_Dec 2019_Feb 2020.csv")
raw_data.columns

Index(['Tweet Id', 'Tweet URL', 'Tweet Posted Time (UTC)', 'Tweet Content',
       'Tweet Type', 'Client', 'Retweets Received', 'Likes Received',
       'Tweet Location', 'Lat', 'Long', 'Tweet Language', 'User Id', 'Name',
       'Screen Name', 'User Bio', 'Verified or Non-Verified', 'Profile URL',
       'Protected or Non-protected', 'User Followers', 'User Following',
       'User Account Creation Date'],
      dtype='object')

In [3]:
reduced_df = raw_data.loc[:, ['Tweet Id',  'Tweet URL', 'Tweet Location', 'Tweet Posted Time (UTC)']]
reduced_df = reduced_df[reduced_df['Tweet Location'].notna()]
#reduced_df['Tweet Posted Month'] = pd.DatetimeIndex(reduced_df['Tweet Posted Time (UTC)']).month 
#reduced_df['Tweet Posted Year'] = pd.DatetimeIndex(reduced_df['Tweet Posted Time (UTC)']).year 
reduced_df.head()

,Tweet Id,Tweet URL,Tweet Location,Tweet Posted Time (UTC)
1,"""1233417742520332290""",https://twitter.com/LAMofficial/status/1233417...,Los Angeles CA USA,2/28/2020 15:44
2,"""1233417741027225602""",https://twitter.com/mitchellvii/status/1233417...,"Miami, FL",2/28/2020 15:44
3,"""1233417699264356357""",https://twitter.com/HelenKennedy/status/123341...,NYC and the North Fork,2/28/2020 15:44
4,"""1233417674274807808""",https://twitter.com/W7VOA/status/1233417674274...,James S. Brady Press Briefing Room,2/28/2020 15:44
6,"""1233417416123834369""",https://twitter.com/GHS/status/123341741612383...,NY/Delhi/Rio/Beijing/Nairobi/Joburg/London,2/28/2020 15:43


In [4]:
#cleaning column values
#columns = ['Tweet Id',  'Tweet URL', 'Tweet Location', 'Lat', 'Long'] #list for filter
#search_for = ['north carolina', 'charlotte', 'n.c.', 'raleigh'] 

filtered_df = reduced_df[reduced_df['Tweet Location'].str.contains('north carolina|charlotte|raleigh|N[.]C[.]', 
                                                                   case = False)] #insensitive  
len(filtered_df)
filtered_df.head()

,Tweet Id,Tweet URL,Tweet Location,Tweet Posted Time (UTC)
441,"""1233400471458586625""",https://twitter.com/BGEA/status/12334004714585...,"Charlotte, N.C.",2/28/2020 14:36
977,"""1233381598285963264""",https://twitter.com/HuffmanForNC/status/123338...,"North Carolina, USA",2/28/2020 13:21
1357,"""1232304248404180993""",https://twitter.com/sbjsbd/status/123230424840...,"Charlotte, NC",2/25/2020 14:00
4419,"""1230546010562154498""",https://twitter.com/DukeHealth/status/12305460...,"Durham, North Carolina",2/20/2020 17:33
4827,"""1229542796874387456""",https://twitter.com/Aubreyz8/status/1229542796...,"North Carolina, USA",2/17/2020 23:07


In [6]:
base_url = "https://maps.googleapis.com/maps/api/geocode/json?" 
params = {"key": gkey}

#places = filtered_df['Tweet Location'].tolist() 
#api_param = [w.replace(' ', '+') for w in places] 

 
for index, row in filtered_df.iterrows():
    
    place = row['Tweet Location'] 
 
    params['address'] = f"{place}" 
    
    resp_data = requests.get(base_url, params = params).json()

    try: 
        filtered_df.loc[index, 'Latitude'] = resp_data['results'][0]['geometry']['location']['lat']
        filtered_df.loc[index, 'Longitude'] = resp_data['results'][0]['geometry']['location']['lng']
        #lats_list.append(resp_data['results'][0]['geometry']['location']['lat'])  
        #lngs_list.append(resp_data['results'][0]['geometry']['location']['lng'])  
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        pass
    
filtered_df.head() 

/Users/znurtokatli/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/znurtokatli/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Missing field/result... skipping.


,Tweet Id,Tweet URL,Tweet Location,Tweet Posted Time (UTC),Latitude,Longitude
0,"""1233400471458586625""",https://twitter.com/BGEA/status/12334004714585...,"Charlotte, N.C.",2/28/2020 14:36,35.227087,-80.843127
1,"""1233381598285963264""",https://twitter.com/HuffmanForNC/status/123338...,"North Carolina, USA",2/28/2020 13:21,35.759573,-79.019300
2,"""1232304248404180993""",https://twitter.com/sbjsbd/status/123230424840...,"Charlotte, NC",2/25/2020 14:00,35.227087,-80.843127
3,"""1230546010562154498""",https://twitter.com/DukeHealth/status/12305460...,"Durham, North Carolina",2/20/2020 17:33,35.994033,-78.898619
4,"""1229542796874387456""",https://twitter.com/Aubreyz8/status/1229542796...,"North Carolina, USA",2/17/2020 23:07,35.759573,-79.019300


In [16]:
filtered_df = filtered_df.dropna()
filtered_df.reset_index(inplace = True) 
del filtered_df['index'] 
filtered_df

,Tweet Id,Tweet URL,Tweet Location,Tweet Posted Time (UTC),Latitude,Longitude
0,"""1233400471458586625""",https://twitter.com/BGEA/status/12334004714585...,"Charlotte, N.C.",2020-02-28 14:36:00,35.227087,-80.843127
1,"""1233381598285963264""",https://twitter.com/HuffmanForNC/status/123338...,"North Carolina, USA",2020-02-28 13:21:00,35.759573,-79.019300
2,"""1232304248404180993""",https://twitter.com/sbjsbd/status/123230424840...,"Charlotte, NC",2020-02-25 14:00:00,35.227087,-80.843127
3,"""1230546010562154498""",https://twitter.com/DukeHealth/status/12305460...,"Durham, North Carolina",2020-02-20 17:33:00,35.994033,-78.898619
4,"""1229542796874387456""",https://twitter.com/Aubreyz8/status/1229542796...,"North Carolina, USA",2020-02-17 23:07:00,35.759573,-79.019300
...,...,...,...,...,...,...
91,"""1223363702885441537""",https://twitter.com/mom_grateful/status/122336...,North Carolina at present,2020-01-31 21:53:00,35.759573,-79.019300
92,"""1223363546081136640""",https://twitter.com/aschaefe29/status/12233635...,"North Carolina, USA",2020-01-31 21:52:00,35.759573,-79.019300
93,"""1223363324601143302""",https://twitter.com/Plebo5/status/122336332460...,"North Carolina, USA",2020-01-31 21:52:00,35.759573,-79.019300
94,"""1216803653073264640""",https://twitter.com/VIGILINT_/status/121680365...,"Raleigh, NC",2020-01-13 19:26:00,35.779590,-78.638179


In [17]:
# save to csv 
filtered_df.to_csv("../Outputs/tweets.csv", encoding = "UTF-8", index = False, header = True) 

In [18]:
filtered_df.describe()

,Latitude,Longitude
count,96.000000,96.000000
mean,35.747031,-79.160481
std,0.457643,0.909313
min,35.214403,-80.947315
25%,35.759573,-79.019300
50%,35.759573,-79.019300
75%,35.759573,-79.019300
max,38.029306,-75.466516


In [19]:
filtered_df.loc[:, 'Tweet Posted Time (UTC)'] = pd.to_datetime(filtered_df['Tweet Posted Time (UTC)']) 

agg_df = filtered_df.groupby([filtered_df['Tweet Posted Time (UTC)'].dt.year, 
                              filtered_df['Tweet Posted Time (UTC)'].dt.month]).count()  #for heat map
agg_df

/Users/znurtokatli/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Tweet Id  Tweet URL  \
Tweet Posted Time (UTC) Tweet Posted Time (UTC)                        
2020                    1                              64         64   
                        2                              32         32   

                                                 Tweet Location  \
Tweet Posted Time (UTC) Tweet Posted Time (UTC)                   
2020                    1                                    64   
                        2                                    32   

                                                 Tweet Posted Time (UTC)  \
Tweet Posted Time (UTC) Tweet Posted Time (UTC)                            
2020                    1                                             64   
                        2                                             32   

                                                 Latitude  Longitude  
Tweet Posted Time (UTC) Tweet Posted Time (UTC)                       
2020                    1                              64         64  
                        2                              32         32

In [20]:
locations = filtered_df[["Latitude", "Longitude"]].astype(float)


In [21]:
#maps  
gmaps.configure(api_key = gkey)
map_figure = gmaps.figure()

layer = gmaps.heatmap_layer(locations, weights = filtered_df, dissipating = False, #weights???
                            max_intensity = 100, point_radius = 1)

map_figure.add_layer(layer)
map_figure



TraitError: Tweet Id is not a valid weight. Weights must be floats